In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from numpy.random import uniform
from torch_geometric.loader import DataLoader
from gnn_tracking.preprocessing.point_cloud_builder import PointCloudBuilder
from gnn_tracking.utils.plotting import GraphPlotter
from gnn_tracking.utils.plotting import PointCloudPlotter
from gnn_tracking.models.track_condensation_networks import PointCloudTCN, GraphTCN

In [ ]:
from gnn_tracking.utils.plotting import EventPlotter

# we'll use n_evts * n_sectors = 640 graphs
n_evts, n_sectors = 10, 64
indir='/tigress/jdezoort/codalab/train_1'
event_plotter = EventPlotter(indir=indir)
event_plotter.plot_ep_rv_uv(evtid=21289)

In [ ]:
! mkdir ~/data/gnn_tracking

In [ ]:
from pathlib import Path

# build point clouds for each sector in the pixel layers only
pc_builder = PointCloudBuilder(indir='/tigress/jdezoort/codalab/train_1', outdir=str(Path("~/data/gnn_tracking/point_clouds").expanduser()),
                               n_sectors=n_sectors, pixel_only=True, redo=False, measurement_mode=False, thld=0.9)
pc_builder.process(n=10, verbose=False)

In [ ]:
# each point cloud is a PyG Data object 
point_cloud = pc_builder.data_list[0]
good = ((point_cloud.sector>-1) & (point_cloud.particle_id>0) &
        (point_cloud.pt > 0.5))


In [ ]:
point_cloud

In [ ]:


# visualize the secto
# rs in each event and an overlapped ('extended') sector
pc_plotter = PointCloudPlotter(str(Path("~/data/gnn_tracking/point_clouds").expanduser()),
                               n_sectors=pc_builder.n_sectors)
pc_plotter.plot_ep_rv_uv_all_sectors(21289)
# pc_plotter.plot_ep_rv_uv_with_boundary(21289, 18,
#                                        pc_builder.sector_di,
#                                        pc_builder.sector_ds)

In [ ]:
! mkdir /home/kl5675/data/gnn_tracking/graphs

In [ ]:
# we can build graphs on the point clouds using geometric cuts

from gnn_tracking.graph_construction.graph_builder import GraphBuilder
graph_builder = GraphBuilder(str(Path("~/data/gnn_tracking/point_clouds").expanduser()), str(Path("~/data/gnn_tracking/graphs").expanduser()), redo=False)
graph_builder.process(verbose=True, n=None)

In [ ]:
! ls  /home/kl5675/data/gnn_tracking/

In [ ]:


# the graph plotter shows the true and false edges constructed by the builder

graph_plotter = GraphPlotter()
graph = graph_builder.data_list[0]
print(graph)
evtid, s = graph.evtid.item(), graph.s.item()

# takes a minute to run, but cool visual!
#graph_plotter.plot_rz(graph_builder.data_list[0], 
#          f'event{evtid}_s{s}')

In [ ]:
# use cuda (gpu) if possible, otherwise fallback to cpu
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'Utilizing {device}')

# use reference graph to get relevant dimensions 
g = graph_builder.data_list[0]
node_indim = g.x.shape[1]
edge_indim = g.edge_attr.shape[1]
hc_outdim = 2 # output dim of latent space  

# partition graphs into train, test, val splits
graphs = graph_builder.data_list
n_graphs = len(graphs)
rand_array = uniform(low=0, high=1, size=n_graphs)
train_graphs = [g for i, g in enumerate(graphs) if (rand_array<=0.7)[i]]
test_graphs = [g for i, g in enumerate(graphs) if ((rand_array>0.7) & (rand_array<=0.9))[i]]
val_graphs = [g for i, g in enumerate(graphs) if (rand_array>0.9)[i]]

# build graph loaders
params = {'batch_size': 1, 'shuffle': True, 'num_workers': 2}
train_loader = DataLoader(list(train_graphs), **params)
params = {'batch_size': 1, 'shuffle': False, 'num_workers': 2}
test_loader = DataLoader(list(test_graphs), **params)
val_loader = DataLoader(list(val_graphs), **params)
loaders = {'train': train_loader, 'test': test_loader,
           'val': val_loader}
print('Loader sizes:', [(k, len(v)) for k, v in loaders.items()])

# set up a model and trainer
model = GraphTCN(node_indim, edge_indim, hc_outdim, hidden_dim=64)
trainer = GraphTCNTrainer(model=model, loaders=loaders)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
trainer.train()